In [17]:
!pip install xmltodict

In [8]:
!pip install opencv-python

     ---------------------------------------- 38.2/38.2 MB 8.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
tf.__version__

'2.10.1'

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2
import xmltodict
import json

In [4]:
img = cv2.imread('./Detection/helmet_img_608/helmet_img_0.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img.shape

(608, 608, 3)

In [5]:
img = cv2.imread('./Detection/helmet_img/helmet_img_0.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img.shape

(416, 416, 3)

In [3]:
pd.read_xml('../Semi_Data/annotations/BikesHelmets0.xml')

,folder,filename,width,height,depth,segmented,name,pose,truncated,occluded,difficult,bndbox
0,images,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
1,None,BikesHelmets0.png,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
2,None,None,400.0,267.0,3.0,NaN,None,None,NaN,NaN,NaN,NaN
3,None,None,NaN,NaN,NaN,0.0,None,None,NaN,NaN,NaN,NaN
4,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN
5,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN
6,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN
7,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN


In [7]:
pip install xmltodict

  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
# import json
# import xmltodict

# ## xml 파일을 json 파일로 변환

# for i in range(766) :
#     try :
#         with open(f"../Semi_Data/annotations/BikesHelmets{i}.xml",'r') as f:
#             xmlString = f.read()
            
#         jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
        
#         with open(f"../Semi_Data/annotation/BikesHelmets{i}.json", 'w') as f:
#             f.write(jsonString)
#     except :
#         pass

In [10]:
import cv2

# 이미지 로드
image = cv2.imread('../Semi_Data/images/BikesHelmets0.png')
# image.shape
# 이미지 크기 조정
image = cv2.resize(image, (416, 416))

cv2.imwrite('./practice.png', image)
# 픽셀 값 정규화
image = image.astype('float32')/255.0

# YOLO 모델 입력 데이터 형식으로 변환
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = np.expand_dims(image, axis = 0) # 이미지 배열의 차원을 하나 추가해서 (1, height, width, channels) 형식으로 변환

(267, 400, 3)

In [43]:
import cv2
import json

for i in range(766) :
    try :
        # json 파일에서 bndbox 정보 추출
        with open(f'../Semi_Data/annotation/BikesHelmets{i}.json', 'r') as f:
            label = json.load(f)

        bndboxes = []
        for obj in label['annotation']['object']:
            bndbox = obj['bndbox']
            xmin = int(bndbox['xmin'])
            ymin = int(bndbox['ymin'])
            xmax = int(bndbox['xmax'])
            ymax = int(bndbox['ymax'])
            bndboxes.append((xmin, ymin, xmax, ymax))

        # 이미지 로드
        img = cv2.imread(f'../Semi_Data/images/BikesHelmets{i}.png')

        # 경계 상자 그리기
        for bndbox in bndboxes:
            cv2.rectangle(img, (bndbox[0], bndbox[1]), (bndbox[2], bndbox[3]), (0, 255, 0), 2)

        # 이미지 저장
        cv2.imwrite(f'../Semi_Data/bounded_images/BikeHelmets{i}.png', img) # 저장할 경로 입력
    except :
        try :
            with open(f'../Semi_Data/annotation/BikesHelmets{i}.json', 'r') as f:
                label = json.load(f)
            
            bndboxes = []
            bndbox = label['annotation']['object']['bndbox']
            xmin = int(bndbox['xmin'])
            ymin = int(bndbox['ymin'])
            xmax = int(bndbox['xmax'])
            ymax = int(bndbox['ymax'])
            bndboxes.append((xmin, ymin, xmax, ymax))
            
            img = cv2.imread(f'../Semi_Data/images/BikesHelmets{i}.png')

        # 경계 상자 그리기
            for bndbox in bndboxes:
                cv2.rectangle(img, (bndbox[0], bndbox[1]), (bndbox[2], bndbox[3]), (0, 255, 0), 2)

            # 이미지 저장
            cv2.imwrite(f'../Semi_Data/bounded_images/BikeHelmets{i}.png', img) # 저장할 경로 입력
            
        except :
            pass
        


In [95]:
def save_data(file_path, image_save_path, label_save_path_json, label_save_path_xml, save_file_index) :
    
    idx = save_file_index
    
    # file path에 있는 파일들 리스트로 담기
    file_list = os.listdir(file_path)
    
    # 파일이름에 .png가 있으면 image_list에 담기
    image_list = [i for i in file_list if '.png' in i]
    
    # 파일이름에 .xml이 있으면 label_list에 담기
    label_list = [i for i in file_list if '.xml' in i]
    
    
    
    for image,label in zip(image_list, label_list) : 
        # image 로드
        # img = cv2.imread(file_path + image)
        
        # 이미지 저장
        # cv2.imwrite(image_save_path + f'helmet_img_{idx}.png', img) # 저장할 경로 입력
        
        # xml 파일 json으로 바꿔서 저장하기    
    
        with open(file_path + label,'r',encoding='UTF8') as f:
            xmlString = f.read()
        
        # with open(label_save_path_xml + f'helmet_label_{idx}.xml', 'w') as f:
        #     f.write(xmlString)
        
        jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
        
        with open(label_save_path_json + label[:-4]+ '.json', 'w') as f:
            f.write(jsonString)
    
        idx += 1
    
    print(f'마지막 파일 idx는 {idx - 1}입니다.')
   

In [96]:
save_data('./Detection/608_data/','./Detection/helmet_img_608/','./Detection/608_json/','./Detection/helmet_label_xml_608/',0)

마지막 파일 idx는 6289입니다.


In [ ]:
# save_data('./with_helmet_add_416/','./helmet_img/', './helmet_label_json/', './helmet_label_xml/', 1784)

In [26]:
# save_data('./Data/helmet datas/With Helmet/','./Data/helmet_image/', './Data/helmet_label/', 0)

마지막 파일 idx는 99입니다.


In [27]:
# save_data('./Data/helmet datas/Without Helmet/','./Data/helmet_image/', './Data/helmet_label/', 100)

마지막 파일 idx는 199입니다.


In [47]:
# save_data('./Data/helmet_YB/With_Helmet_image/','./Data/helmet_image/', './Data/helmet_label/', 200)

마지막 파일 idx는 299입니다.


In [48]:
# save_data('./Data/helmet_YB/Without_Helmet_image/','./Data/helmet_image/', './Data/helmet_label/', 300)

마지막 파일 idx는 399입니다.


In [49]:
# save_data('./Data/kaggle_image_resize/','./Data/helmet_image/', './Data/helmet_label/', 400)

마지막 파일 idx는 1123입니다.


In [50]:
# save_data('./Data/withouthelmet_resize/','./Data/helmet_image/', './Data/helmet_label/', 1124)

마지막 파일 idx는 1326입니다.


In [18]:
# save_data('./data/without_helmet_add_resize/','./data/helmet_image/', './data/helmet_label/', 1327)

마지막 파일 idx는 1783입니다.


In [20]:
# 이미지를 자르고 저장하는 함수
def cut_image(json_path, image_path) : 
    
    with_idx = 0
    without_idx = 0
    json_file = os.listdir(json_path)
    image_file = os.listdir(image_path)
    n = len(image_file)
    
    for i in range(n) :
        try :
            # json 파일에서 bndbox 정보 추출
            with open(json_path + json_file[i], 'r') as f:
                label = json.load(f)
            
            bndboxes = []
                            
            for obj in label['annotation']['object']:
                bndbox = obj['bndbox']
                img_l = obj['name']
                xmin = int(bndbox['xmin'])
                ymin = int(bndbox['ymin'])
                xmax = int(bndbox['xmax'])
                ymax = int(bndbox['ymax'])
                bndboxes.append((xmin, ymin, xmax, ymax, img_l))
            # print('clear')
            # 이미지 자르기
            for bndbox in bndboxes:
                try : 
                    # 이미지 로드
                    img = Image.open(image_path + image_file[i])
                    image = img.crop((bndbox[0], bndbox[1], bndbox[2], bndbox[3]))
                    # 이미지 저장
                    if bndbox[4] == 'With Helmet' :
                        image.save(f'./data/with_helmet/with_helmet_{with_idx}.png') # 저장할 경로 입력
                        with_idx += 1
                    
                    else :
                        image.save(f'./data/without_helmet/without_helmet_{without_idx}.png')
                        without_idx += 1
                except :
                    print(json_file[i])
        except :
            try :
                with open(json_path + json_file[i], 'r') as f:
                    label = json.load(f)
                
                bndbox = label['annotation']['object']['bndbox']
                img_l = label['annotation']['object']['name']
                xmin = int(bndbox['xmin'])
                ymin = int(bndbox['ymin'])
                xmax = int(bndbox['xmax'])
                ymax = int(bndbox['ymax'])
            
                
                # 이미지 자르기
                
                # 이미지 로드
                img = Image.open(image_path + image_file[i])
                image = img.crop((xmin, ymin, xmax, ymax))
                
                if img_l == 'With Helmet' :
                    image.save(f'./data/with_helmet/with_helmet_{with_idx}.png') # 저장할 경로 입력
                    with_idx += 1
                
                else :
                    image.save(f'./data/without_helmet/without_helmet_{without_idx}.png')
                    without_idx += 1
            except :
                pass

    

In [21]:
image_label = cut_image('./data/helmet_label/', './data/helmet_image/')

In [ ]:
fiel_list = os.listdir('./Data/cut_image/')

In [91]:
# 이미지 사이즈 조절하여 저장 하는 함수

def img_resize_save(img_path, save_path, image_shape) :
    img_list = os.listdir(img_path)
    count = 0
    
    if not os.path.exists(save_path) :
        os.mkdir(save_path)
        
    for img in img_list :
        image = cv2.imread(img_path + img)
        
        image = cv2.resize(image, image_shape)
        
        cv2.imwrite(save_path+img, image)
        count += 1


In [12]:
img_resize_save('./Detection/add_data/', './Detection/add_data_resize/', (608, 608))

In [6]:
img_resize_save('./data/with_helmet_resize_128/','./data/with_helmet_resize_64/',(64, 64))

In [7]:
img_resize_save('./data/without_helmet_resize_128/','./data/without_helmet_resize_64/',(64, 64))

In [59]:
img_resize_save('./Data/cut_image/', './Data/cut_image_resize/')


In [13]:
img_resize_save('./data/with_helmet_gen/', './data/with_helmet_resize_128/', (128, 128))

In [24]:
img_resize_save('./data/without_helmet_add_gen/', './data/without_helmet_resize_128/', (128, 128))

In [61]:
file_list = os.listdir('./Data/cut_image_resize/')
df = pd.DataFrame([file_list, image_label])

In [63]:
df = df.T

In [65]:
df.columns = ['file_name', 'label']
df

,file_name,label
0,cut_helmet_0.png,With Helmet
1,cut_helmet_1.png,With Helmet
2,cut_helmet_10.png,With Helmet
3,cut_helmet_100.png,Without Helmet
4,cut_helmet_1000.png,Without Helmet
...,...,...
2064,cut_helmet_995.png,Without Helmet
2065,cut_helmet_996.png,Without Helmet
2066,cut_helmet_997.png,Without Helmet
2067,cut_helmet_998.png,With Helmet


In [66]:
df.to_csv('./label.csv', index = False)

In [2]:
def BW_image(img_path, save_path) :
    file_list = os.listdir(img_path)
    for img in file_list :
        color_image = Image.open(img_path + img)
        
        gray_image = color_image.convert('L')
        
        gray_image.save(save_path + img)

In [3]:
BW_image('./data/with_helmet_resize/', './data/with_helmet_BW/')

In [4]:
BW_image('./data/without_helmet_resize/', './data/without_helmet_BW/')

In [2]:
def save_data_1(file_path, label_save_path_xml, label_save_path_json) :
    # file path에 있는 파일들 리스트로 담기
    file_list = os.listdir(file_path)
    
    for label in file_list : 
        
        try :
            with open(file_path + label,'r',encoding='UTF8') as f:
                xmlString = f.read()
        
        except :
            with open(file_path + label,'r') as f:
                xmlString = f.read()
        
        xmlString = xmlString.replace('With_Helmet', 'With Helmet').replace('Without_Helmet', 'Without Helmet')
        
        with open(label_save_path_xml + label, 'w', encoding='UTF8') as f:
            f.write(xmlString)
            
        jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
        
        with open(label_save_path_json + label[:-4]+'.json', 'w') as f:
            f.write(jsonString)

In [9]:
img = cv2.imread('./Detection/helmet_img/helmet_img_1.png')
img = cv2.resize(img, (608,608))
cv2.imwrite('./Detection/train.png',img)

ratio = 608/408

with open('./Detection/helmet_label_xml/helmet_label_1.xml','r',encoding='UTF8') as f:
    xmlString = f.read()
xmlString
with open('./Detection/helmet_label_json/helmet_label_1.json', 'r') as f:
    label = json.load(f)

bndboxes = []
                
bndbox = label['annotation']['object']['bndbox']
img_l = label['annotation']['object']['name']
xmin = int(bndbox['xmin']) 
ymin = int(bndbox['ymin'])
xmax = int(bndbox['xmax'])
ymax = int(bndbox['ymax'])

xmin_new = int(xmin * ratio)
ymin_new = int(ymin * ratio)
xmax_new = int(xmax * ratio)
ymax_new = int(ymax * ratio)

xmlString = xmlString.replace(f'<xmin>{str(xmin)}', f'<xmin>{str(xmin_new)}')
xmlString = xmlString.replace(f'<ymin>{str(ymin)}', f'<ymin>{str(ymin_new)}')
xmlString = xmlString.replace(f'<xmax>{str(xmax)}', f'<xmax>{str(xmax_new)}')
xmlString = xmlString.replace(f'<ymax>{str(ymax)}', f'<ymax>{str(ymax_new)}')

with open('./Detection/train.xml', 'w', encoding='UTF8') as f:
    f.write(xmlString)


In [10]:
image = Image.open('./Detection/train.png')
image = image.crop((xmin_new, ymin_new, xmax_new, ymax_new))
image.save('./Detection/cut.png')

In [7]:
xmlString

'<annotation>\n\t<folder>wh</folder>\n\t<filename>10.png</filename>\n\t<path>C:\\Users\\SBAUser\\Desktop\\helmet datas\\wh\\10.png</path>\n\t<source>\n\t\t<database>Unknown</database>\n\t</source>\n\t<size>\n\t\t<width>416</width>\n\t\t<height>416</height>\n\t\t<depth>3</depth>\n\t</size>\n\t<segmented>0</segmented>\n\t<object>\n\t\t<name>With Helmet</name>\n\t\t<pose>Unspecified</pose>\n\t\t<truncated>0</truncated>\n\t\t<difficult>0</difficult>\n\t\t<bndbox>\n\t\t\t<xmin>143</xmin>\n\t\t\t<ymin>33</ymin>\n\t\t\t<xmax>264</xmax>\n\t\t\t<ymax>214</ymax>\n\t\t</bndbox>\n\t</object>\n</annotation>\n'

In [97]:
def img_resize_label(img_path, img_save_path, xml_path, json_path, xml_save_path, image_shape, ratio) :
    img_resize_save(img_path, img_save_path, image_shape)
    
    xml_list = os.listdir(xml_path)
    json_list = os.listdir(json_path)
    
    for x, j in zip(xml_list, json_list) : 
        
        try :
            with open(xml_path + x,'r',encoding='UTF8') as f:
                xmlString = f.read()
        
        except :
            with open(xml_path + x,'r') as f:
                xmlString = f.read()
                
        
        try :
            # json 파일에서 bndbox 정보 추출
            with open(json_path + j , 'r') as f:
                file = json.load(f)
            
            
                            
            for obj in file['annotation']['object']:
                bndbox = obj['bndbox']
                xmin = int(bndbox['xmin'])
                ymin = int(bndbox['ymin'])
                xmax = int(bndbox['xmax'])
                ymax = int(bndbox['ymax'])
                
                xmin_new = int(xmin * ratio)
                ymin_new = int(ymin * ratio)
                xmax_new = int(xmax * ratio)
                ymax_new = int(ymax * ratio)
                
                xmlString = xmlString.replace(f'<xmin>{str(xmin)}', f'<xmin>{str(xmin_new)}')
                xmlString = xmlString.replace(f'<ymin>{str(ymin)}', f'<ymin>{str(ymin_new)}')
                xmlString = xmlString.replace(f'<xmax>{str(xmax)}', f'<xmax>{str(xmax_new)}')
                xmlString = xmlString.replace(f'<ymax>{str(ymax)}', f'<ymax>{str(ymax_new)}')
                
            with open(xml_save_path + x, 'w', encoding='UTF8') as f:
                f.write(xmlString)
                
        except :
            try :
                with open(json_path + j , 'r') as f:
                    file = json.load(f)
                
                bndbox = file['annotation']['object']['bndbox']
                xmin = int(bndbox['xmin'])
                ymin = int(bndbox['ymin'])
                xmax = int(bndbox['xmax'])
                ymax = int(bndbox['ymax'])
            
                xmin_new = int(xmin * ratio)
                ymin_new = int(ymin * ratio)
                xmax_new = int(xmax * ratio)
                ymax_new = int(ymax * ratio)
            
                xmlString = xmlString.replace(f'<xmin>{str(xmin)}', f'<xmin>{str(xmin_new)}')
                xmlString = xmlString.replace(f'<ymin>{str(ymin)}', f'<ymin>{str(ymin_new)}')
                xmlString = xmlString.replace(f'<xmax>{str(xmax)}', f'<xmax>{str(xmax_new)}')
                xmlString = xmlString.replace(f'<ymax>{str(ymax)}', f'<ymax>{str(ymax_new)}')
                
                with open(xml_save_path + x, 'w', encoding='UTF8') as f:
                    f.write(xmlString)
                    
            except :
                print(f'{x}파일은 이상해')
    
    
    
        
    

In [98]:
img_path = './Detection/608_img/'
img_save_path = './Detection/416_img/'
xml_path = './Detection/608_xml/'
json_path = './Detection/608_json/'
xml_save_path = './Detection/416_xml/'
image_shape = (416,416)
ratio = 416/608
img_resize_label(img_path, img_save_path, xml_path, json_path, xml_save_path, image_shape, ratio)

In [ ]:
def save_data_1(file_path, label_save_path_xml) :
    # file path에 있는 파일들 리스트로 담기
    file_list = os.listdir(file_path)
    
    for label in file_list : 
        
        try :
            with open(file_path + label,'r',encoding='UTF8') as f:
                xmlString = f.read()
        
        except :
            with open(file_path + label,'r') as f:
                xmlString = f.read()
        
        xmlString = xmlString.replace('<width>608', '<width>416').replace('<height>608', '<height>416')
        
        with open(label_save_path_xml + label, 'w', encoding='UTF8') as f:
            f.write(xmlString)

In [2]:
with open('./Detection/helmet_label_xml_416/helmet_label_0.xml','r',encoding='UTF8') as f:
    xmlString = f.read()
xmlString

'<annotation>\n\t<folder>wh</folder>\n\t<filename>10.png</filename>\n\t<path>C:\\Users\\SBAUser\\Desktop\\helmet datas\\wh\\10.png</path>\n\t<source>\n\t\t<database>Unknown</database>\n\t</source>\n\t<size>\n\t\t<width>416</width>\n\t\t<height>416</height>\n\t\t<depth>3</depth>\n\t</size>\n\t<segmented>0</segmented>\n\t<object>\n\t\t<name>With Helmet</name>\n\t\t<pose>Unspecified</pose>\n\t\t<truncated>0</truncated>\n\t\t<difficult>0</difficult>\n\t\t<bndbox>\n\t\t\t<xmin>143</xmin>\n\t\t\t<ymin>33</ymin>\n\t\t\t<xmax>264</xmax>\n\t\t\t<ymax>214</ymax>\n\t\t</bndbox>\n\t</object>\n</annotation>\n'

In [5]:
file_list = os.listdir('./Detection/helmet_label_xml_608/')
file_list[0].replace('label', 'img')

'helmet_img_0.xml'

In [6]:
def save_data_2(file_path, label_save_path_xml) :
    # file path에 있는 파일들 리스트로 담기
    file_list = os.listdir(file_path)
    
    for label in file_list : 
        
        try :
            with open(file_path + label,'r',encoding='UTF8') as f:
                xmlString = f.read()
        
        except :
            with open(file_path + label,'r') as f:
                xmlString = f.read()
        
        with open(label_save_path_xml + label.replace('label','img'), 'w', encoding='UTF8') as f:
            f.write(xmlString)

In [88]:
save_data_2('./Detection/helmet_label_xml_608/','./Detection/608_data/')

In [23]:
txt_file = os.listdir('./Detection/416_data/')
txt_file = [i for i in txt_file if 'txt' in i]
txt_file = txt_file[1:]
len(txt_file)

6290

In [24]:
txt_file[3][11:-4]

'100'

In [25]:
len(txt_file)

6290

In [26]:
c = []
b = list(range(6291))
for i in txt_file :
    a = int(i[11:-4])
    b.remove(a)
    
print(b)
# b = 
# for i in c :

        

[6290]


In [33]:
b = list(range(2001))

In [35]:
b.remove(1)

In [28]:
file_path = './Detection/416_data/'
txt_file = os.listdir(file_path)
txt_file = [i for i in txt_file if 'txt' in i]

for i in txt_file :
    with open(file_path + i,'r') as f:
        txtString = f.readlines()
    for line in txtString :
        if str(line[0]) == '2' :
            print(i)

In [29]:
with open(file_path + txt_file[-1], 'r') as f:
    txtString = f.readlines()
txtString

['0 0.497596 0.292067 0.081731 0.146635\n',
 '0 0.730769 0.262019 0.086538 0.177885\n']

In [82]:
img_list = os.listdir('./Detection/add_data_resize/')
xml_list = os.listdir('./Detection/add_data_resize/')

img_list = [i for i in img_list if 'png' in i]
xml_list = [i for i in xml_list if 'xml' in i]



In [83]:
len(img_list), len(xml_list)

(3771, 3771)

In [73]:
img_list = img_list + ['add_data6_9999.png', 'add_data7_9999.png']

In [76]:
img_list[0][10:-4]

'0'

In [84]:
for i, x in zip(img_list, xml_list) :
    if i[10:-4] != x[10:-4] :
        print(i)
        print(x)
        break


'2.0.0+cu117'

In [30]:
file_list = os.listdir('./helmet_data/')

In [31]:
file_list = [i[:-4] for i in file_list if '.xml' in i]
file_list

['helmet_img_0',
 'helmet_img_1',
 'helmet_img_10',
 'helmet_img_100',
 'helmet_img_1000',
 'helmet_img_1001',
 'helmet_img_1002',
 'helmet_img_1003',
 'helmet_img_1004',
 'helmet_img_1005',
 'helmet_img_1006',
 'helmet_img_1007',
 'helmet_img_1009',
 'helmet_img_1010',
 'helmet_img_1011',
 'helmet_img_1012',
 'helmet_img_1013',
 'helmet_img_1014',
 'helmet_img_1015',
 'helmet_img_1016',
 'helmet_img_1020',
 'helmet_img_1021',
 'helmet_img_1022',
 'helmet_img_1023',
 'helmet_img_1025',
 'helmet_img_1026',
 'helmet_img_1027',
 'helmet_img_1028',
 'helmet_img_1029',
 'helmet_img_1030',
 'helmet_img_1032',
 'helmet_img_1033',
 'helmet_img_1035',
 'helmet_img_1036',
 'helmet_img_1039',
 'helmet_img_104',
 'helmet_img_1041',
 'helmet_img_1042',
 'helmet_img_1043',
 'helmet_img_1044',
 'helmet_img_1045',
 'helmet_img_1046',
 'helmet_img_1048',
 'helmet_img_1049',
 'helmet_img_105',
 'helmet_img_1050',
 'helmet_img_1052',
 'helmet_img_1053',
 'helmet_img_1054',
 'helmet_img_1055',
 'helmet_im

In [32]:
txt_file = os.listdir('./Detection/416_data/')
txt_file = [i for i in txt_file if 'txt' in i]
for file in file_list :
    if file+'.txt' in txt_file :
        with open('./Detection/416_data/' + file + '.txt', 'r') as f :
            text = f.read()
        
        with open('./helmet_data/' + file + '.txt', 'w') as f :
            f.write(text)
